zi2zi: Master Chinese Calligraphy with Conditional Adversarial Networks


### 步驟一與步驟二

#### 連接 google drive 並確認 workspace.

In [ ]:
import os
from google.colab import drive
drive.mount('/content/drive')

workspace = 'ct2022s'

# Try to aceess your workspace, or create it.
try:
  os.chdir(os.path.join('/content/drive/MyDrive/', workspace))
except:
  os.mkdir(os.path.join('/content/drive/MyDrive/', workspace))
  os.chdir(os.path.join('/content/drive/MyDrive/', workspace))

#### 確認環境並下載 zi2zi 專案📡

In [1]:
%tensorflow_version 1.x
!pip install scipy==1.2.1
!git clone https://github.com/t110598009/zi2zi.git
# Acess zi2zi
os.chdir('zi2zi')

UsageError: Line magic function `%tensorflow_version` not found.


### 步驟三: 訓練前處理 (產生資料)

To avoid IO bottleneck, preprocessing is necessary to pickle your data into binary and persist in memory during training.

First run the below command to get the font images:  
```
python font2img.py --src_font=font/'source font' --dst_font=font/'Target font' --charset=CN_T  --sample_count=1000 --sample_dir='Your sample dir' --label=0  --filter=1 --shuffle=1
```

#### 建立欲訓練或驗證的圖片

In [ ]:
# 建立 sample 資料夾
!mkdir image_train
!mkdir image_val

In [ ]:
!python font2img.py --src_font=font/NotoSansTC-Black.otf --dst_font=font/setofont.ttf --charset=TWTrain  --sample_count=1000 --sample_dir=image_train --label=1 --filter=1 --shuffle=1
!python font2img.py --src_font=font/NotoSansTC-Black.otf --dst_font=font/setofont.ttf --charset=TWVal  --sample_count=670 --sample_dir=image_val --label=1 --filter=1 --shuffle=0


#### 建立訓練、驗證資料 object
After obtaining all images, run package.py to pickle the images and their corresponding labels into binary format:

跑完後會得到 train.obj and val.obj 在 save_dir 資料夾。

In [ ]:
!python package.py --dir=image_train --save_dir=experiment/data --split_ratio=0.1

In [ ]:
!python package.py --dir=image_val --save_dir=experiment/data/val --split_ratio=1

### 訓練模型

In [ ]:
!python train.py --experiment_dir=experiment --experiment_id=1 --batch_size=16 --lr=0.001 --epoch=40 --sample_steps=50 --schedule=20 --L1_penalty=100 --Lconst_penalty=15

### 推論 (或稱驗證)

In [ ]:
!python infer.py --model_dir=experiment/checkpoint/experiment_1_batch_16 --batch_size=16 --source_obj=experiment/data/val/val.obj --embedding_ids=1 --save_dir=experiment/infer_1

### ttf-downsize

In [ ]:
def generate_ttf_downsize(self):
  from google.colab import files
  import os
  txt_data = ''
  with open(f'./charset/{txt_file_name}', 'r', encoding='utf-8') as f:
    txt_data = f.read()
  glyphs = []
  for i in txt_data:
    if not i in glyphs:
      glyphs.append(i)
  codes = []
  for i in glyphs:
    unicode = hex(ord(i))[2:]
    while len(unicode) < 4:
      unicode = '0' + unicode
    codes.append('U+'+unicode)
  with open('codes.txt', 'w', encoding='utf-8') as f:
    f.write('\n'.join(codes))
  cmd = 'pyftsubset \"%s\" --output-file=\"%s\" --unicodes-file=\"%s\"' % (
      os.path.join('./font', dd.value),
      os.path.join('./font', dd.value.replace('.ttf', '-downsize.ttf')),
      os.path.join(os.getcwd(), 'codes.txt')
  )
  os.system(cmd)
  files.download(os.path.join('./font', dd.value.replace('.ttf', '-downsize.ttf')))

Generate txt file

In [ ]:
import json
cjk = json.load(open("./charset/cjk.json"))
output = cjk["twVal"]
txt_file_name = "val.txt"
with open(f'./charset/{txt_file_name}', 'w', encoding='utf-8') as f:
  for word in output:
    f.write(word)

Install Dependencies

In [ ]:
!pip install fonttools

Upload ttf file (Optional section that it could takes few minutes)

In [ ]:
# # ctrl + A and ctrl + K + U to uncomment this scetion
# from google.colab import files
# os.chdir('font')
# ttf_file = files.upload()
# ttf_file_name = list(ttf_file.keys())[0]
# os.chdir("..")

Select font from 雲端內 ct2022s/zi2zi/font 資料夾

In [ ]:
import ipywidgets as widgets
import os

dropdown_list = ['......'] + [f for f in os.listdir('./font') if f.split('.')[-1] == 'ttf']

dd = widgets.Dropdown(options=dropdown_list, value='......')

display(dd)
dd.observe(generate_ttf_downsize, names= 'value')

#Acknowledgements
原作者:
* [zi2zi](https://github.com/kaonashi-tyc/zi2zi) by kaonashi-tyc(original author)

Code derived and rehashed from:

* [pix2pix-tensorflow](https://github.com/yenchenlin/pix2pix-tensorflow) by yenchenlin
* [Domain Transfer Network](https://github.com/yunjey/domain-transfer-network) by yunjey
* [ac-gan](https://github.com/buriburisuri/ac-gan) by buriburisuri
* [dc-gan](https://github.com/carpedm20/DCGAN-tensorflow) by carpedm20
* [origianl pix2pix torch code](https://github.com/phillipi/pix2pix) by phillipi